# APP

In [18]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import joblib


class RobotIKSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("6‑DOF Robot IK Solver")
        self.root.geometry("500x900")
        self.root.resizable(False, False)

        # Load model and scaler safely
        try:
            self.model = tf.keras.models.load_model("IK_6in_6out_FINAL.h5", compile=False)
            self.scaler = joblib.load("input_scaler.pkl")
        except Exception as e:
            raise RuntimeError(f"Error loading model/scaler: {e}")

        # Background
        self._set_background("robot.jpg")

        # Main panel
        self.panel = tk.Frame(self.root, bg="#111111")
        self.panel.place(relx=0.5, rely=0.5, anchor="center", width=400, height=750)

        # Title
        tk.Label(self.panel, text="Robot IK Solver",
                 font=("Arial", 22, "bold"), bg="#111111", fg="#00eaff").pack(pady=20)

        # Variables (entries display degrees for angles, raw values for XYZ)
        self.val_x = tk.StringVar(value="0.00")
        self.val_y = tk.StringVar(value="0.00")
        self.val_z = tk.StringVar(value="0.00")
        self.val_roll = tk.StringVar(value="0.00")   # degrees in entry
        self.val_pitch = tk.StringVar(value="0.00")  # degrees in entry
        self.val_yaw = tk.StringVar(value="0.00")    # degrees in entry

        # Sliders + Entries
        self.slider_x, self.entry_x = self._create_slider_with_entry("X Position", -300, 300, self.val_x, is_angle=False)
        self.slider_y, self.entry_y = self._create_slider_with_entry("Y Position", -300, 300, self.val_y, is_angle=False)
        self.slider_z, self.entry_z = self._create_slider_with_entry("Z Position", -300, 300, self.val_z, is_angle=False)
        self.slider_roll, self.entry_roll = self._create_slider_with_entry("Roll (°)", -np.pi, np.pi, self.val_roll, is_angle=True)
        self.slider_pitch, self.entry_pitch = self._create_slider_with_entry("Pitch (°)", -np.pi, np.pi, self.val_pitch, is_angle=True)
        self.slider_yaw, self.entry_yaw = self._create_slider_with_entry("Yaw (°)", -np.pi, np.pi, self.val_yaw, is_angle=True)

        # Initialize sliders to 0
        for s in (self.slider_x, self.slider_y, self.slider_z, self.slider_roll, self.slider_pitch, self.slider_yaw):
            s.set(0.0)

        # Result box
        self.result_text = tk.StringVar()
        tk.Label(self.panel, textvariable=self.result_text,
                 font=("Consolas", 14), bg="#111111", fg="#00ffcc",
                 justify="center").pack(pady=10)

        # Footer
        tk.Label(self.root, text="Developed by Ahmed | Neural IK System",
                 font=("Arial", 10), bg="#111111", fg="white").place(relx=0.5, rely=0.98, anchor="center")

        # Style
        self._set_button_style()

        # First prediction
        self._predict_and_update_plot()

    # ---------------- Background ----------------
    def _set_background(self, image_path):
        try:
            bg_img = Image.open(image_path).resize((500, 900), Image.LANCZOS)
            bg_photo = ImageTk.PhotoImage(bg_img)
            bg_label = tk.Label(self.root, image=bg_photo)
            bg_label.image = bg_photo
            bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except FileNotFoundError:
            print("Background image not found. Continuing without it.")

    # ---------------- Slider + Entry ----------------
    def _create_slider_with_entry(self, label, from_, to_, variable, is_angle=False):
        tk.Label(self.panel, text=label, bg="#111111", fg="white",
                 font=("Arial", 12)).pack(pady=(10, 0))

        frame = tk.Frame(self.panel, bg="#111111")
        frame.pack(pady=5, fill="x", padx=40)

        # Slider stores radians for angles, raw value for XYZ
        slider = ttk.Scale(
            frame, from_=from_, to=to_, orient="horizontal",
            command=lambda val: self._update_from_slider(val, variable, is_angle)
        )
        slider.pack(side="left", fill="x", expand=True)

        # Entry shows degrees for angles, raw value for XYZ
        entry = ttk.Entry(frame, textvariable=variable, width=8)
        entry.pack(side="right", padx=5)

        entry.bind("<Return>", lambda e: self._update_from_entry(entry, slider, from_, to_, variable, is_angle))
        entry.bind("<FocusOut>", lambda e: self._update_from_entry(entry, slider, from_, to_, variable, is_angle))

        return slider, entry

    def _update_from_slider(self, val, variable, is_angle):
        v = float(val)
        # Display degrees for angles, raw value for XYZ
        display_val = np.degrees(v) if is_angle else v
        variable.set(f"{display_val:.2f}")
        self._predict_and_update_plot()

    def _update_from_entry(self, entry, slider, from_, to_, variable, is_angle):
        try:
            typed = float(variable.get())
            # Convert typed degrees to radians for slider if angle
            slider_val = np.radians(typed) if is_angle else typed
            # Clamp within slider range
            if from_ <= slider_val <= to_:
                slider.set(slider_val)
                self._predict_and_update_plot()
        except ValueError:
            # Clear invalid text, keep previous
            entry.delete(0, tk.END)

    # ---------------- Prediction ----------------
    def _predict_and_update_plot(self):
        # Sliders return radians for angles; model expects radians (good)
        x = self.slider_x.get()
        y = self.slider_y.get()
        z = self.slider_z.get()
        roll = self.slider_roll.get()
        pitch = self.slider_pitch.get()
        yaw = self.slider_yaw.get()

        # Update entry displays (degrees for angles, raw for XYZ)
        self.val_x.set(f"{x:.2f}")
        self.val_y.set(f"{y:.2f}")
        self.val_z.set(f"{z:.2f}")
        self.val_roll.set(f"{np.degrees(roll):.2f}")
        self.val_pitch.set(f"{np.degrees(pitch):.2f}")
        self.val_yaw.set(f"{np.degrees(yaw):.2f}")

        # Prepare input and predict
        inp = np.array([[x, y, z, roll, pitch, yaw]])
        inp_scaled = self.scaler.transform(inp)
        pred = self.model.predict(inp_scaled, verbose=0)[0]

        # Convert output to degrees for display
        pred_deg = np.degrees(pred)
        self.result_text.set("\n".join([f"q{i+1} = {angle:.2f}°" for i, angle in enumerate(pred_deg)]))

    # ---------------- Style ----------------
    def _set_button_style(self):
        style = ttk.Style()
        style.theme_use("clam")
        style.configure(
            "Rounded.TButton",
            font=("Arial", 14, "bold"),
            padding=10,
            background="#00eaff",
            foreground="black",
            borderwidth=0
        )
        style.map("Rounded.TButton", background=[("active", "#00bcd4")])


if __name__ == "__main__":
    root = tk.Tk()
    app = RobotIKSolverApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Ahmed Hussien\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\Ahmed Hussien\AppData\Local\Temp\ipykernel_8820\4051660724.py", line 92, in <lambda>
    command=lambda val: self._update_from_slider(val, variable, is_angle)
                        ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ahmed Hussien\AppData\Local\Temp\ipykernel_8820\4051660724.py", line 110, in _update_from_slider
    self._predict_and_update_plot()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\Ahmed Hussien\AppData\Local\Temp\ipykernel_8820\4051660724.py", line 150, in _predict_and_update_plot
    self.result_text.set("\n".join([f"q{i+1} = {angle:.2f}°" for i, angle in enumerate(pred_deg)]))
    ^^^^^^^^^^^^^^^^
AttributeError: 'RobotIKSolverApp' object has no attribute 'result_text'
Exception 

# COMPERED PREDICT DATA WITH TRUE DATA

In [5]:
import os
import numpy as np
import pandas as pd
DATA_PATH = "robot_dataset_filtered"  
def load_dataset(path):
    df = pd.read_csv(path)
    cols_in = ["x", "y", "z", "roll", "pitch", "yaw"]
    cols_out = ["q1","q2","q3","q4","q5","q6"]

    X = df[cols_in]
    Y = df[cols_out]
    return X, Y

    X, Y = load_dataset(DATA_PATH)
X[["roll", "pitch", "yaw"]] = np.degrees(X[["roll", "pitch", "yaw"]])
Y[["q1","q2","q3","q4","q5","q6"]] = np.degrees(Y[["q1","q2","q3","q4","q5","q6"]])
X,Y

(              x        y         z        roll      pitch         yaw
 0       257.520   44.853   56.3860  -43.179245 -60.567369  119.507537
 1       274.780  -54.385    5.0973  126.617943  30.500835  -20.637940
 2       178.000 -183.180  -67.0610  -21.080836 -47.326887    7.250208
 3       -52.205 -201.890 -109.2600  -15.467569 -27.609117   97.202290
 4        16.470 -213.850  124.1700  -43.201018 -50.102294   33.000650
 ...         ...      ...       ...         ...        ...         ...
 154956   83.181  201.260 -102.1000   44.531426 -19.871322 -139.612626
 154957  -84.557 -175.800 -119.9800   34.429607 -35.278730  118.865824
 154958  142.190  185.100  -89.5740   16.085790  18.854322 -138.300553
 154959 -121.890 -209.570  -77.6450  -17.257489  29.562330   60.670501
 154960 -200.790  153.760   79.9380  167.091682  23.665449   97.895569
 
 [154961 rows x 6 columns],
                 q1         q2         q3          q4         q5         q6
 0        38.240922  70.067009  39.377670 